In [55]:
def collect_samples(game, node, p, p_not, M_r, B_vp, B_s):
    if game.is_terminal(node):
        x = game.util(node, p)
        print("at node", node, "util for", p, "is", x)
        return x
    elif game.P(node) == p:
        I = game.I(node, p)
        A = game.A(node)
        sigma = calculate_strategy(I, A, M_r[p])
        v_a = np.zeros(3)
        for a in A:
            v_a[a] = collect_samples(game, game.take(node, a), p, p_not, M_r, B_vp, B_s)
        v_s = np.dot(v_a, sigma)
        d = v_a - v_s
        print("at node", node, "for", p)
        print("v_s is", v_s)
        print("sigma is", sigma)
        print("v_a is", v_a)
        print("d is", d)
        B_vp.add(I, d)
        return v_s
    elif game.P(node) == p_not:
        I = game.I(node, p_not)
        A = game.A(node)
        sigma = calculate_strategy(I, A, M_r[p_not])
        B_s.add(I, sigma)
        try:
            a = np.random.choice(3, p=sigma)
        except ValueError:
            a = np.random.choice(3, p=sigma/sigma.sum())
        return collect_samples(game, game.take(node, a), p, p_not, M_r, B_vp, B_s)
    else:
        return collect_samples(game.deal(), "B", p, p_not, M_r, B_vp, B_s)

def calculate_strategy(I, A, model):
    sigma = np.zeros(3)
    d = model.predict(I)[0, A]
    d_plus = np.clip(d, 0, None)
    if d_plus.sum() > 0:
        sigma[A] = d_plus/d_plus.sum()
        return sigma
    else:
        sigma[A[np.argmax(d)]] = 1
        return sigma


In [56]:
''' objects.py'''
import tensorflow as tf
import numpy as np
from operator import itemgetter

class buffer:
    def __init__(self):
        self.list = [[],[],[],[]]
        self.size = 0
        self.count = 0

    def add(self, input, output):
        self.list[0].append(input[0])
        self.list[1].append(input[1])
        self.list[2].append(input[2])
        self.list[3].append(output)
        self.size += 1
        self.count += 1
        return self.list

    def set(self):
        self.count = 0

    def sample(self, indices):
        return (itemgetter(*indices)(self.list[0]), itemgetter(*indices)(self.list[1]), itemgetter(*indices)(self.list[2]), itemgetter(*indices)(self.list[3]))

def connect(input, weights, biases, activations):
    layer = input
    for W, b, activation in zip(weights, biases, activations):
        layer = tf.matmul(layer, W) + b
        if activation is not None:
            layer = activation(layer)
    return layer

class model:
    def __init__(self, name, softmax=True):
        self.name = name
        self.graph = tf.Graph()
        with self.graph.as_default():
            with tf.variable_scope(name):
                self.output_ph = tf.placeholder(dtype=tf.float32, shape=[None, 3])

                b1d = tf.get_variable(name='b1d', shape=[16], initializer=tf.constant_initializer(0.))
                b2d = tf.get_variable(name='b2d', shape=[16], initializer=tf.constant_initializer(0.))
                b3d = tf.get_variable(name='b3d', shape=[3], initializer=tf.constant_initializer(0.))
                W2d = tf.get_variable(name='W2d', shape=[16, 16], initializer=tf.contrib.layers.xavier_initializer())
                W3d = tf.get_variable(name='W3d', shape=[16, 3], initializer=tf.contrib.layers.xavier_initializer())

                self.input_pha = tf.placeholder(dtype=tf.float32, shape=[None, 1])
                W0a = tf.get_variable(name='W0a', shape=[1, 32], initializer=tf.contrib.layers.xavier_initializer())
                W1a = tf.get_variable(name='W1a', shape=[32, 16], initializer=tf.contrib.layers.xavier_initializer())
                b0a = tf.get_variable(name='b0a', shape=[32], initializer=tf.constant_initializer(0.))
                activations = [tf.nn.relu, None]
                weights = [W0a, W1a]
                biases = [b0a, b1d]
                outputa = connect(self.input_pha, weights, biases, activations)

                self.input_phb = tf.placeholder(dtype=tf.float32, shape=[None, 3])
                W0b = tf.get_variable(name='W0b', shape=[3, 32], initializer=tf.contrib.layers.xavier_initializer())
                W1b = tf.get_variable(name='W1b', shape=[32, 16], initializer=tf.contrib.layers.xavier_initializer())
                b0b = tf.get_variable(name='b0b', shape=[32], initializer=tf.constant_initializer(0.))
                weights = [W0b, W1b]
                biases = [b0b, b1d]
                outputb = connect(self.input_phb, weights, biases, activations)

                self.input_phc = tf.placeholder(dtype=tf.float32, shape=[None, 3])
                W0c = tf.get_variable(name='W0c', shape=[3, 32], initializer=tf.contrib.layers.xavier_initializer())
                W1c = tf.get_variable(name='W1c', shape=[32, 16], initializer=tf.contrib.layers.xavier_initializer())
                b0c = tf.get_variable(name='b0c', shape=[32], initializer=tf.constant_initializer(0.))
                weights = [W0c, W1c]
                biases = [b0c, b1d]
                outputc = connect(self.input_phc, weights, biases, activations)

                inputd = outputa + outputb + outputc
                if softmax:
                    activations = [tf.nn.relu, tf.nn.softmax]
                else:
                    activations = [tf.nn.relu, None]
                weights = [W2d, W3d]
                biases = [b2d, b3d]
                self.output_pred = connect(inputd, weights, biases, activations)

                self.mse = tf.reduce_mean(0.5 * tf.square(self.output_pred - self.output_ph))
                self.opt = tf.train.AdamOptimizer().minimize(self.mse)

                self.sess = tf.Session()
                self.sess.run(tf.global_variables_initializer())
                self.saver = tf.train.Saver()

    def predict(self, inputs):
        return self.sess.run(self.output_pred, feed_dict={self.input_pha: [inputs[0]], self.input_phb: [inputs[1]], self.input_phc: [inputs[2]]})

    def train(self, B, weights, N_train, N_batch):
        for training_step in range(N_train):
            weights = np.array(weights)
            sample = B.sample(np.random.choice(B.size, N_batch, p=weights/weights.sum()))
            _, mse_run = self.sess.run([self.opt, self.mse], feed_dict={self.input_pha: sample[0], self.input_phb: sample[1], self.input_phc: sample[2], self.output_ph: sample[3]})
            if training_step % 1000 == 0:
                print('%s: %04d mse: %0.3f'%(self.name, training_step, mse_run))

# fold 0; check 1; bet 2.
class game:
    def __init__(self):
        self.tree = {
                    "B": ["", "C", "D"],
                    "C": ["", "E", "F"],
                    "D": ["G", "", "H"],
                    "F": ["J", "", "K"]}
        self.info = {
                    "B": [[0,0,0],[0,0,0]],
                    "C": [[1,0,0],[1,0,0]],
                    "D": [[2,0,0],[1,0,0]],
                    "E": [[1,1,0],[1,1,0]],
                    "F": [[1,2,0],[1,1,0]],
                    "G": [[2,0,0],[1,1,0]],
                    "H": [[2,2,0],[1,1,0]],
                    "J": [[1,2,1],[1,1,1]],
                    "K": [[1,2,2],[1,1,1]]}
        self.available = {
                        "B": [1,2],
                        "C": [1,2],
                        "D": [0,2],
                        "F": [0,2]}

    def deal(self):
        self.cards = np.random.choice(3,(2,1))
        return self

    def util(self, node, p):
        if node == "E":
            return (p == np.argmax(self.cards))*2
        elif node == "J":
            return [0,3][p]
        elif node == "K":
            return (p == np.argmax(self.cards))*4
        elif node == "G":
            return [3,0][p]
        elif node == "H":
            return (p == np.argmax(self.cards))*4
        else:
            return None

    def take(self, node, action):
        return self.tree[node][action]

    def A(self, node):
        return self.available[node]

    def P(self, node):
        if node in ["C", "D"]:
            return 1
        elif node in ["B", "F"]:
            return 0
        else:
            return None

    def is_terminal(self, node):
        if node in ["E", "J", "K", "G", "H"]:
            return True
        else:
            return False

    def I(self, node, p):
        return (self.cards[p], *self.info[node])


In [60]:
G = game()
M_r = (model('p0', True), model('p1', True))
B_vp = buffer()
B_s = buffer()
collect_samples(G, "A", 0, 1, M_r, B_vp, B_s)

at node E util for 0 is 2
at node H util for 0 is 4
at node B for 0
v_s is 3.0
sigma is [0.  0.5 0.5]
v_a is [0. 2. 4.]
d is [-3. -1.  1.]


3.0

In [53]:
print(B_vp.list)
print(G.cards)


[[array([2])], [[0, 0, 0]], [[0, 0, 0]], [array([-3.1262126, -1.1262126,  0.8737874])]]
[[2]
 [1]]


In [18]:
G.is_terminal("A")

False

In [3]:
from objects import *
M = model("state")
M.restore("state")

INFO:tensorflow:Restoring parameters from ./saves/state_model.ckpt


In [13]:
from deep_cfr import *
G = game()
G.cards = np.array([0,1])
I = G.I("C",0)
print(I)
A = G.A("C")
calculate_strategy(I, A, M)

ValueError: Cannot feed value of shape (1,) for Tensor 'state/Placeholder_1:0', which has shape '(?, 1)'

In [10]:
%tb

SystemExit: 2

In [12]:
from tests import *